# Introduction

**No** hand shake or a big **No** to hugs! CoronaVirus is changing the world, people and their perspective. 
From a 2 year old to a 90 year old - everyone is aware of Corona and dealing with it in different ways. When my 5 year old is  yelling in her sleep *"**Germs- big Red Germs- Run***", my mom and dad are washing dishes together recollecting their old days. 

Now the big question is are we going to get our old life back at the end of this Corona Rule or we will have an entirely brand new approach to life? What do you think?

In the mean time, let's try to find out more about the virus or it's devastating skills.

# Install Libraries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime
# Plots
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import seaborn as sns

#
import warnings
warnings.filterwarnings("ignore")
import itertools
import statsmodels.api as sm
#
from statsmodels.tsa.arima_model import ARIMA

# Load Data

In [ ]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
pd.options.display.float_format = '{:,.0f}'.format

In [ ]:
df1 = pd.read_csv("/kaggle/input/2019-coronavirus-dataset-01212020-01262020/2019_nCoV_20200121_20200201.csv")
df2 = pd.read_csv("/kaggle/input/2019-coronavirus-dataset-01212020-01262020/2019_nCoV_20200121_20200205.csv")
df3 = pd.read_csv("/kaggle/input/2019-coronavirus-dataset-01212020-01262020/2019_nCoV_20200121_20200206.csv")
df4 = pd.read_csv("/kaggle/input/2019-coronavirus-dataset-01212020-01262020/2019_nCoV_20200121_20200130.csv")
df5 = pd.read_csv("/kaggle/input/2019-coronavirus-dataset-01212020-01262020/2019_nCoV_20200121_20200127.csv")
df6 = pd.read_csv("/kaggle/input/2019-coronavirus-dataset-01212020-01262020/2019_nCoV_20200121_20200131.csv")
df7 = pd.read_csv("/kaggle/input/2019-coronavirus-dataset-01212020-01262020/2019_nCoV_20200121_20200128.csv")
df_summary = pd.read_csv("/kaggle/input/2019-coronavirus-dataset-01212020-01262020/2019_nC0v_20200121_20200126 - SUMMARY.csv")
df_clean = pd.read_csv("/kaggle/input/2019-coronavirus-dataset-01212020-01262020/2019_nC0v_20200121_20200126_cleaned.csv")

# Explore Data

In [ ]:
print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)
print(df5.shape)
print(df6.shape)
print(df7.shape)
print(df_summary.shape)
print(df_clean.shape)

# Data Cleaning

### 1.Merge Datasets

In [ ]:
df_covid = pd.concat([df1,df2,df3,df4,df5,df6,df7], axis=0)
df_covid = df_covid.sort_values("Last Update")
df_covid.shape

In [ ]:
df_covid.info()

In [ ]:
df_covid.head()

### 2. Rename Columns

In [ ]:
df_covid.rename(columns = {'Province/State':'State'}, inplace = True) 
df_covid.rename(columns = {'Country/Region':'Country'}, inplace = True) 
df_covid.rename(columns = {'Last Update':'last_update'}, inplace = True) 

### 3.Change Column Type

In [ ]:
df_covid.last_update = pd.to_datetime(df_covid.last_update).dt.date.astype(str)

### 4. Check For Missing Values

In [ ]:
df_covid.isnull().sum()

### 5. Replace nan values with zero(0)

In [ ]:
df_covid["Confirmed"] = df_covid.Confirmed.fillna(0)
df_covid["Suspected"] = df_covid.Suspected.fillna(0)
df_covid["Recovered"] = df_covid.Recovered.fillna(0)
df_covid["Death"] = df_covid.Death.fillna(0)

In [ ]:
df_covid.head(2)

**We do not have much information in this dataset. But let's try to best use of whatever information we have.**

# Feature Extraction

### Working with Dates and Cases Count

In [ ]:
df_dates = df_covid.groupby("last_update")[["last_update","Confirmed","Suspected","Recovered","Death"]].sum().reset_index()

In [ ]:
df_dates['last_update'].min(), df_dates['last_update'].max()

# Data Visualization

### Confirmed Cases

In [ ]:
plt.figure(figsize=(15,6))
sns.set_color_codes("pastel")
pl1 = sns.barplot(x="last_update", y=df_dates.sort_index().Confirmed.cumsum(), data=df_dates,
            label="Total", color="b").set_title("Cumulative Confirmed cases of Covid-19")
plt.xticks(rotation=90,horizontalalignment='right',
    fontweight='light',
    fontsize='medium'  )

**Conclusion: With each passing day, the confirmed cases count increased.**

### Recovered Cases

In [ ]:
plt.figure(figsize=(15,6))
sns.set_color_codes("pastel")
pl1 = sns.barplot(x="last_update", y=df_dates.sort_index().Recovered.cumsum(), data=df_dates,
            label="Total", color="g").set_title("Cumulative Recovered cases of Covid-19")
plt.xticks(rotation=90,horizontalalignment='right',
    fontweight='light',
    fontsize='medium'  )

**Conclusion: As more and more people were getting affected, some people were getting recovered too**

### Deaths

In [ ]:
plt.figure(figsize=(15, 6))
sns.set_color_codes("pastel")
pl1 = sns.barplot(x="last_update", y=df_dates.sort_index().Death.cumsum(), data=df_dates,
            label="Total", color="r").set_title("Cumulative Deaths cases of Covid-19")
plt.xticks(rotation=90,horizontalalignment='right',
    fontweight='light',
    fontsize='medium'  )

**Conclusion: Deaths due to coronavirus increased with each day**

### Compare Confirmed Cases And Deaths Count due to Covid-19

In [ ]:
plt.figure(figsize=(15, 6))
X = np.arange(1)
plt.bar(X + 0.00, df_dates.Confirmed.sum(), color = 'yellow', width = 0.25, label="Confirmed")
plt.bar(X + 0.25, df_dates.Death.sum(), color = 'red', width = 0.25, label = "Death")
plt.title("Confirmed vs Deaths")
plt.grid(True)
plt.legend()
plt.show()

**Conclusion: Lets try to be positive. Deaths were very less compare to the confimed cases.**

### Compare Recovered And Deaths Count due to Covid-19

In [ ]:
plt.figure(figsize=(15, 6))
X = np.arange(1)
plt.bar(X + 0.00, df_dates.Recovered.sum(), color = 'g', width = 0.25, label = "Recovered")
plt.bar(X + 0.25, df_dates.Death.sum(), color = 'r', width = 0.25, label = "Death")
plt.title("Deaths vs Recovered Cases")
plt.grid(True)
plt.legend()
plt.show()

**Conclusion: It's a relief to know in that short period of time in Jan and Feb, Recovered cases were more than Deaths.**

### Most Affected Countries

In [ ]:
plt.figure(figsize=(15, 6))
df_covid.Country.value_counts().plot("bar")
plt.grid(True)

**Conclusion: As expected, China is the most affected country, followed by United States. During that small time period, Italy was far behind**

### Most Affected States in United States

In [ ]:
plt.figure(figsize=(15, 6))
df_covid[df_covid.Country == "United States"]["State"].value_counts().plot("bar")
plt.grid(True)

**Conclusion: Washington was at the top in that period.**

# Forecasting Model

In this dataset, we have some dates and the confirmed cases in those dates. Let's try to build a Forecasting Model to predict more confirmed cases of Covid-19 in the near future.

In [ ]:
df_train = df_covid.groupby("last_update")[["last_update","Confirmed"]].mean().reset_index()
df_train = df_train.set_index('last_update')
df_train.index

In [ ]:
df_train.head(2)

In [ ]:
#y = df_train['Confirmed']
df_train.plot(figsize=(15, 6))
plt.show()

The above plot shows clearly that the confirmed cases of Covid-19 increased with each passing day. 

# Fit a Model

Here, we will use the previous values of the time series to predict its future values. Thus, it is an example of **Univariate Time Series Forecasting**. Let's use ARIMA(Auto Regressive Integrated Moving Average) for the forcasting.

An ARIMA model is characterized by 3 terms: p, d, q

where,

p: the order of the AR term
q: the order of the MA term
d: the number of differencing required to make the time series stationary

Let's use a “grid search” to iteratively explore different combinations of parameters. For each combination of parameters, we will fit a new ARIMA model and assess its overall quality.

Finally, we will select the optimal set of parameters that yields the best performance. 

In [ ]:
# Define the p, d and q parameters to take any value between 0 and 2
p = d = q = range(0, 2)

# Generate all different combinations of p, q and q triplets
pdq = list(itertools.product(p, d, q))

# Fit a Model for each param value
for param in pdq:
    try:
        #print(param)
        model = ARIMA(df_train,order=param )
        results = model.fit(disp=0)
        print('ARIMA - AIC:',(param, results.aic))
    except:
        continue


The above output suggests that ARIMA(0, 1, 0) yields the lowest AIC value of 180.2962. Therefore we should consider this to be optimal option.

### Run the Best Fit

In [ ]:
ts_model = ARIMA(df_train,order=(0, 1, 0))
ts_results = ts_model.fit(disp=1)
print(ts_results.summary())

### Plot The Model

In [ ]:
# Plot residual errors
residuals = pd.DataFrame(ts_results.resid)
fig, ax = plt.subplots(1,2)
residuals.plot(title="Residuals", ax=ax[0], figsize=(10,5))
residuals.plot(kind='kde', title='Density', ax=ax[1],figsize=(10,5))
plt.show()

In [ ]:
# Actual vs Fitted
ts_results.plot_predict(dynamic = False)
plt.show()

I have set dynamic=False. That means, the in-sample lagged values are used for prediction.
That is, the model gets trained up until the previous value to make the next prediction. This can make the fitted forecast and actuals look artificially good.

So, we seem to have a decent ARIMA model. But is it actually the best?

### Predictions

In [ ]:
pred = pd.DataFrame(ts_results.predict(start = '2020-01-22', end = '2020-02-05', typ = 'levels'))
pred = pred.reset_index()
pred.columns = ['Date','Prediction']
prediction = pred.Prediction
prediction

In [ ]:
df_train = df_train.reset_index()
actual = df_train.Confirmed
actual

In [ ]:
mape = np.mean(np.abs(prediction - actual)/np.abs(actual))  # MAPE
rmse = np.mean((prediction - actual)**2)**.5  # RMSE
print(mape,rmse)

Well, it looks like, it is not a very good Model. But, it is just a start!